In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import json

from rich import print
from python.analysis import Master, BeamParticleSelection, PFOSelection, Plots, shower_merging, vector, Tags, cross_section, EventSelection

import awkward as ak
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from particle import Particle
from scipy.optimize import curve_fit


shower_merging.SetPlotStyle(extend_colors = True, dpi = 140)

In [ ]:
config = "work/2023/cex_analysis_2GeV_config.json"
if not config:
    mc = Master.Data("work/ROOTFiles/PDSPAnalyser/PDSPProd4a_MC_2GeV_reco1_sce_datadriven_v1_ntuple_v09_41_00_03.root", nTuple_type = Master.Ntuple_Type.PDSP)
    data = Master.Data("work/ROOTFiles/PDSPAnalyser/PDSPProd4_data_2GeV_reco2_ntuple_v09_42_03_01.root", nTuple_type = Master.Ntuple_Type.PDSP)

    beam_quality_fits = {
        "mc" : "work/2023/PDSPProd4a_MC_2GeV_reco1_sce_datadriven_v1_ntuple_v09_41_00_03/PDSPProd4a_MC_2GeV_reco1_sce_datadriven_v1_ntuple_v09_41_00_03_fit_values.json",
        "data" : "work/2023/PDSPProd4_data_2GeV_reco2_ntuple_v09_42_03_01/PDSPProd4_data_2GeV_reco2_ntuple_v09_42_03_01_fit_values.json"
    }
else:
    args = cross_section.ApplicationArguments.ResolveConfig(cross_section.LoadConfiguration(config))
    mc = Master.Data(args.mc_file, nTuple_type = Master.Ntuple_Type.PDSP, nEvents = -1)
    data = Master.Data(args.data_file, nTuple_type = Master.Ntuple_Type.PDSP, nEvents = -1)
    beam_quality_fits = {"mc" : args.mc_beam_quality_fit, "data" : args.data_beam_quality_fit}

samples = {"mc" : mc, "data": data}


# for f in beam_quality_fits:
#     with open(beam_quality_fits[f], "r") as file:
#         beam_quality_fits[f] = json.load(file)

def FilterSamples(samples : dict, masks : dict, PFO : bool = False):
    for s in samples:
        if PFO:
            samples[s].Filter([masks[s]])
        else:
            samples[s].Filter([masks[s]], [masks[s]])

In [ ]:
Plots.PlotTagged(mc.recoParticles.beam_endPos.z, Tags.GenerateTrueBeamParticleTags(mc), data2 = data.recoParticles.beam_endPos.z, x_range = [-100, 700], norm = True, bins = 50, x_label = "beam end position z (cm)", stacked = True, alpha = 0.5)
Plots.PlotTagged(mc.recoParticles.beam_endPos.z, Tags.GenerateTrueBeamParticleTags(mc), data2 = data.recoParticles.beam_endPos.z, x_range = [-100, 700], norm = True, bins = 50, x_label = "beam end position z (cm)", stacked = False, alpha = 0.5)
# Plots.PlotTagged(mc.trueParticles.endPos.z[:, 0], Tags.GenerateTrueBeamParticleTags(mc), x_range = [-100, 700], norm = True, bins = 50, x_label = "true beam end position z (cm)")

In [ ]:
event_normalisation = ak.count(data.eventNum) / ak.count(mc.eventNum)
pfo_normalisation = ak.count(data.recoParticles.number) / ak.count(mc.recoParticles.number)
Plots.PlotTagged(mc.recoParticles.energy, Tags.GenerateTrueParticleTags(mc), data2 = data.recoParticles.energy, y_scale = "linear", norm = event_normalisation, x_range = [900, 1000])

# Beam Particle Selection

In [ ]:
masks = {
    "mc" : BeamParticleSelection.PiBeamSelection(samples["mc"]),
    "data" : BeamParticleSelection.PiBeamSelection(samples["data"], use_beam_inst = True)
}

counts = Tags.GenerateTrueBeamParticleTags(samples["mc"])
for i in counts:
    counts[i] = ak.sum(counts[i].mask)

bar_data = []
for tag in counts:
    bar_data.extend([tag]*counts[tag])
Plots.PlotBar(bar_data, xlabel = "true particle ID")

FilterSamples(samples, masks)

In [ ]:
masks = {s : BeamParticleSelection.CaloSizeCut(samples[s]) for s in samples}
FilterSamples(samples, masks)

In [ ]:
masks = {s : BeamParticleSelection.PandoraTagCut(samples[s]) for s in samples}

if data is None:
    Plots.PlotBar(samples["mc"])
else:
    Plots.PlotBarComparision(samples["mc"].recoParticles.beam_pandora_tag, samples["data"].recoParticles.beam_pandora_tag, label_1 = "MC", label_2 = "Data", xlabel = "pandora beam tag", fraction = True, barlabel = True)

FilterSamples(samples, masks)

In [ ]:
def norm(x, mu, sigma):
    return (x - mu) / sigma

# Beam quality cuts
for i in ["x", "y", "z"]:
    Plots.PlotTagged(mc.recoParticles.beam_startPos_SCE[i], Tags.GenerateTrueBeamParticleTags(mc), data2 = data.recoParticles.beam_startPos_SCE[i], x_label = i + "(cm)", y_scale = "log", norm = True)

#* dxy cut
dxy = {s : (norm(samples[s].recoParticles.beam_startPos_SCE.x, beam_quality_fits[s]["mu_x"], beam_quality_fits[s]["sigma_x"])**2 + norm(samples[s].recoParticles.beam_startPos_SCE.y, beam_quality_fits[s]["mu_y"], beam_quality_fits[s]["sigma_y"])**2)**0.5 for s in samples}
masks = {s : dxy[s] < 3 for s in samples}

Plots.PlotTagged(dxy["mc"], Tags.GenerateTrueBeamParticleTags(mc), data2 = np.array(dxy["data"]), bins = 50, x_label = "$dxy$ (cm)", y_scale = "log", x_range = [0, 5], norm = True)
Plots.DrawCutPosition(3, arrow_length = 1, face = "left")

[print(f"dxy cut: {BeamParticleSelection.CountMask(masks[s])}") for s in masks]
FilterSamples(samples, masks)


In [ ]:
deltas_mc = {}
deltas_data = {}
for i in ["x", "y", "z"]:
    deltas_mc[i] = norm(samples["mc"].recoParticles.beam_startPos_SCE[i], beam_quality_fits["mc"][f"mu_{i}"], beam_quality_fits["mc"][f"sigma_{i}"])
    deltas_data[i] = norm(samples["data"].recoParticles.beam_startPos_SCE[i], beam_quality_fits["data"][f"mu_{i}"], beam_quality_fits["data"][f"sigma_{i}"])
    Plots.PlotTagged(deltas_mc[i], Tags.GenerateTrueBeamParticleTags(mc), data2 = np.array(deltas_data[i]), bins = 50, x_range = [-10, 10], y_scale = "log", x_label = f"$(" + i + "- \mu_{"+ i + "} )/ \sigma_{" + i + "}$", ncols = 3, norm = True)
    Plots.DrawCutPosition(-3, arrow_length = 1, face = "right")
    Plots.DrawCutPosition(3, arrow_length = 1, face = "left")


masks = {
    "mc" : (deltas_mc["z"] > -3) & (deltas_mc["z"] < 3),
    "data" : (deltas_data["z"] > -3) & (deltas_data["z"] < 3)
}

FilterSamples(samples, masks)

In [ ]:
beam_dir = {s : vector.normalize(vector.sub(samples[s].recoParticles.beam_endPos_SCE, samples[s].recoParticles.beam_startPos_SCE)) for s in samples}

beam_dir_mu = {s : vector.normalize(vector.vector(beam_quality_fits[s]["mu_dir_x"], beam_quality_fits[s]["mu_dir_y"], beam_quality_fits[s]["mu_dir_z"])) for s in beam_quality_fits}

beam_costh = {s : vector.dot(beam_dir[s], beam_dir_mu[s]) for s in beam_dir_mu}

Plots.PlotTagged(beam_costh["mc"], Tags.GenerateTrueBeamParticleTags(mc), data2 = np.array(beam_costh["data"]), x_label = "$\cos(\\theta)$", y_scale = "log", bins = 50, x_range = [0.9, 1], norm = True)
Plots.DrawCutPosition(0.95, arrow_length = 0.02)

masks = {s : beam_costh[s] > 0.95 for s in samples}
FilterSamples(samples, masks)

In [ ]:
Plots.PlotTagged(samples["mc"].recoParticles.beam_endPos_SCE.z, Tags.GenerateTrueBeamParticleTags(samples["mc"]), data2 = samples["data"].recoParticles.beam_endPos.z, x_label = "Beam end position z (cm)", norm = args.norm, bins = 50, y_scale = "log", ncols = 1)
Plots.DrawCutPosition(220, face = "left", arrow_length = 50)

masks = {s : BeamParticleSelection.APA3Cut(samples[s]) for s in samples}

FilterSamples(samples, masks)

In [ ]:
# michel score cut

scores = {}
for s in samples:
    score = ak.where(samples[s].recoParticles.beam_nHits != 0, samples[s].recoParticles.beam_michelScore / samples[s].recoParticles.beam_nHits, -999)
    masks[s] = BeamParticleSelection.MichelScoreCut(samples[s])
    scores[s] = score

Plots.PlotTagged(scores["mc"], Tags.GenerateTrueBeamParticleTags(samples["mc"]), data2 = np.array(scores["data"]), x_range = (0, 1), y_scale = "log", bins = 50, x_label = "michel score", ncols = 3, norm = args.norm)
Plots.DrawCutPosition(0.55, face = "left")

FilterSamples(samples, masks)

In [ ]:
median = {s : PFOSelection.Median(samples[s].recoParticles.beam_dEdX) for s in samples}

Plots.PlotTagged(median["mc"], Tags.GenerateTrueBeamParticleTags(samples["mc"]), data2 = median["data"], y_scale = "log", x_range = [0, 10], x_label = "median $dE/dX$ (MeV/cm)", norm = True)
Plots.DrawCutPosition(2.4, face = "left", arrow_length = 2)

masks = {s : BeamParticleSelection.MedianDEdXCut(samples[s]) for s in samples}
FilterSamples(samples, masks)

In [ ]:
a = {"mc" : "mc_arguments", "data" : "data_arguments"}
masks = {s : BeamParticleSelection.BeamScraperCut(samples[s], args.beam_selection[a[s]]["BeamScraperCut"]["KE_range"], args.beam_selection[a[s]]["BeamScraperCut"]["fits"], args.beam_selection[a[s]]["BeamScraperCut"]["pdg_hyp"], args.beam_selection[a[s]]["BeamScraperCut"]["cut"]) for s in samples}

nx = {s : norm(samples[s].recoParticles.beam_inst_pos.x, ak.mean(samples[s].recoParticles.beam_inst_pos.x), ak.std(samples[s].recoParticles.beam_inst_pos.x)) for s in samples}
ny = {s : norm(samples[s].recoParticles.beam_inst_pos.y, ak.mean(samples[s].recoParticles.beam_inst_pos.y), ak.std(samples[s].recoParticles.beam_inst_pos.y)) for s in samples}

r = {s : (nx[s]**2 + ny[s]**2)**0.5 for s in samples}

Plots.PlotTagged(r["mc"], data2 = r["data"], tags = Tags.GenerateTrueBeamParticleTags(samples["mc"]), norm = True)
Plots.DrawCutPosition(args.beam_selection[a[s]]["BeamScraperCut"]["cut"], arrow_length = 1, face = "left", arrow_loc = 0.6)

FilterSamples(samples, masks)

In [ ]:
samples["mc"].cutTable.get_table()

In [ ]:
samples["data"].cutTable.get_table()

# Quick Beam Selection

In [ ]:
masks = {}
masks["mc"] = BeamParticleSelection.CreateDefaultSelection(samples["mc"], False, beam_quality_fits["mc"], verbose = False, return_table = False)
masks["data"] = BeamParticleSelection.CreateDefaultSelection(samples["data"], True, beam_quality_fits["data"], verbose = False, return_table = False)
FilterSamples(samples, masks)

masks = {s : PFOSelection.GoodShowerSelection(samples[s]) for s in samples}
FilterSamples(samples, masks, True)

# Good PFO selection

In [ ]:
masks = {s : PFOSelection.GoodShowerSelection(samples[s]) for s in samples}
FilterSamples(samples, masks, True)

# $\pi^{+}$ Particle Selection

In [ ]:
# only for my ntuples
masks = {s : PFOSelection.BeamDaughterCut(samples[s]) for s in samples}
Plots.PlotTagged(samples["mc"].recoParticles.trackScore, Tags.GenerateTrueParticleTags(samples["mc"]), data2 = samples["data"].recoParticles.trackScore, y_scale = "log", x_label = "track score", norm = True)


In [ ]:
Plots.PlotTagged(samples["mc"].recoParticles.trackScore, Tags.GenerateTrueParticleTags(samples["mc"]), data2 = samples["data"].recoParticles.trackScore, y_scale = "linear", bins = 50, ncols = 4, x_label = "track score", norm = True)
Plots.DrawCutPosition(0.5, face = "right")

masks = {s : PFOSelection.TrackScoreCut(samples[s]) for s in samples}
FilterSamples(samples, masks, True)

In [ ]:
Plots.PlotTagged(samples["mc"].recoParticles.nHits, Tags.GenerateTrueParticleTags(samples["mc"]), data2 = samples["data"].recoParticles.nHits, bins = 50, ncols = 2, x_range = [0, 500], norm = True)
# Plots.PlotHist2D(ak.ravel(samples["mc"].trueParticlesBT.completeness), ak.ravel(samples["mc"].recoParticles.nHits), xlabel = "completeness", ylabel = "nHits", y_range = [0, 500], bins = 50)

Plots.PlotHist2DImshowMarginal(ak.ravel(samples["mc"].recoParticles.nHits), ak.ravel(samples["mc"].trueParticlesBT.completeness), ylabel = "completeness", xlabel = "nHits", x_range = [0, 500], bins = 50, norm = "column", c_scale = "linear")
Plots.DrawCutPosition(20, arrow_length = 100, arrow_loc = 0.1, color = "C6")

masks = {s : PFOSelection.NHitsCut(samples[s], 20) for s in samples}
FilterSamples(samples, masks, True)

In [ ]:
median_dEdX = {s : PFOSelection.Median(samples[s].recoParticles.track_dEdX) for s in samples}
Plots.PlotTagged(median_dEdX["mc"], Tags.GenerateTrueParticleTags(samples["mc"]), data2 = median_dEdX["data"], ncols = 2, x_range = [0, 5], x_label = "median $dEdX$", bins = 50, norm = True)

Plots.DrawCutPosition(0.5, arrow_length = 0.5, face = "right")
Plots.DrawCutPosition(2.8, arrow_length = 0.5, face = "left")

masks = {s : PFOSelection.PiPlusSelection(samples[s]) for s in samples}
FilterSamples(samples, masks, True)

In [ ]:
tags = Tags.GenerateTrueParticleTags(samples["mc"])
bar_data = []
for tag in tags:
    bar_data.extend([tag]*ak.sum(tags[tag].mask))

Plots.PlotBar(bar_data, xlabel = "true particle ID")


# photon candidate selection

In [ ]:
Plots.PlotTagged(samples["mc"].recoParticles.emScore, Tags.GenerateTrueParticleTags(samples["mc"]), data2 = samples["data"].recoParticles.emScore, bins = 50, x_range = [0, 1], ncols = 5, x_label = "em score", norm = True)
Plots.DrawCutPosition(0.5)

masks = {s : PFOSelection.EMScoreCut(samples[s], 0.5) for s in samples}
FilterSamples(samples, masks, True)

In [ ]:
Plots.PlotTagged(samples["mc"].recoParticles.nHits, Tags.GenerateTrueParticleTags(samples["mc"]), data2 = samples["data"].recoParticles.nHits, bins = 50, x_label = "number of hits", x_range = [0, 1000], norm = True)
Plots.DrawCutPosition(80, arrow_length = 100)

Plots.PlotHist2DImshowMarginal(ak.ravel(samples["mc"].recoParticles.nHits), ak.ravel(samples["mc"].trueParticlesBT.completeness), bins = 50, y_range = [0, 1],x_range = [0, 1000], ylabel = "completeness", xlabel = "number of hits", norm = "column")
Plots.DrawCutPosition(80, flip = False, arrow_length = 100, color = "C6")

masks = {s : PFOSelection.NHitsCut(samples[s], 80) for s in samples}
FilterSamples(samples, masks, True)

In [ ]:
dist = {s : vector.dist(samples[s].recoParticles.startPos, samples[s].recoParticles.beam_endPos) for s in samples}

Plots.PlotTagged(dist["mc"], Tags.GenerateTrueParticleTags(samples["mc"]), data2 = dist["data"], bins = 31, x_range = [0, 93], x_label = "distance from PFO start to beam end position (cm)", ncols = 4, norm = True, stacked = "ascending", truncate = True)
Plots.DrawCutPosition(3, arrow_length = 30, arrow_loc = 0.7)
Plots.DrawCutPosition(90, face = "left", arrow_length = 30, arrow_loc = 0.7)

masks = {s : PFOSelection.BeamParticleDistanceCut(samples[s], [3, 90]) for s in samples}
FilterSamples(samples, masks, True)

In [ ]:
ip = {s : PFOSelection.find_beam_impact_parameters(samples[s]) for s in samples}

Plots.PlotTagged(ip["mc"], Tags.GenerateTrueParticleTags(samples["mc"]), data2 = ip["data"], x_range = [0, 50], bins = 50, x_label = "impact parameter wrt beam (cm)", norm = args.norm, stacked = "ascending", ncols = 4, truncate = True)
Plots.DrawCutPosition(20, arrow_length = 20, face = "left")

Plots.PlotHist2DImshowMarginal(ak.ravel(ip["mc"]), ak.ravel(samples["mc"].trueParticlesBT.completeness), x_range = [0, 40], y_range = [0, 1], bins = 20, norm = "column", c_scale = "linear", ylabel = "completeness", xlabel = "impact parameter wrt beam (cm)")
Plots.DrawCutPosition(20, arrow_length = 20, face = "left", color = "red")

masks = {s : PFOSelection.BeamParticleIPCut(samples[s], 20) for s in samples}
FilterSamples(samples, masks, True)


In [ ]:
tags = Tags.GenerateTrueParticleTags(samples["mc"])
bar_data = []
for tag in tags:
    bar_data.extend([tag]*ak.sum(tags[tag].mask))

Plots.PlotBar(bar_data, xlabel = "true particle ID")

# $\pi^{0}$ selection

In [ ]:
samples["mc"].cutTable.get_table()
print(samples["mc"].cutTable.get_masks()[0])
print(samples["mc"].cutTable._signatures)
print(samples["mc"].cutTable._end_sig)

print(len(samples["mc"].cutTable.get_masks()[0][-1]))
print(ak.count(samples["mc"].cutTable.get_masks()[0][-1]))
print(ak.sum(samples["mc"].cutTable.get_masks()[0][-1]))

In [ ]:
# samples["mc"].cutTable._get_mask_signature(mask = samples["mc"].cutTable.get_masks()[0][-1], end = True)
mask = samples["mc"].cutTable.get_masks()[0][-1]
print(mask)
flat_array = isinstance(ak.count(mask, axis=0), int)
curr_simul_masks = samples["mc"].cutTable._group_masks(rcount=1)
mask = curr_simul_masks[0]
for m in curr_simul_masks[1:]:
    mask = mask[m]
if flat_array:
    end_sig = (ak.num(mask, axis=0), -1)
else:
    end_sig = (ak.num(mask, axis=0), ak.sum(mask))
print(end_sig)

print(curr_simul_masks)
# print(mask)
print(flat_array)

In [ ]:
photonCandidates = {s : PFOSelection.InitialPi0PhotonSelection(samples[s]) for s in samples}

In [ ]:
n_photons = {s : ak.sum(photonCandidates[s], -1) for s in samples}

scale = ak.count(n_photons["data"]) / ak.count(n_photons["mc"])

n_photons_scaled = []
u, c = np.unique(n_photons["mc"], return_counts = True)
for i, j in zip(u, c):
    n_photons_scaled.extend([i]* int(scale * j))

Plots.PlotBarComparision(n_photons_scaled, n_photons["data"], xlabel = "number of $\pi^{0}$ photon candidates", label_1 = "MC", label_2 = "Data", barlabel = False)

masks = {s : n_photons[s] == 2 for s in samples}

print(len(masks["mc"]))
print(ak.count(masks["mc"]))
print(ak.sum(masks["mc"]))


FilterSamples(samples, masks)
photonCandidates = {s : photonCandidates[s][masks[s]] for s in samples}

In [ ]:
shower_pairs = {s : Master.ShowerPairs(samples[s], shower_pair_mask = photonCandidates[s]) for s in samples}

#* energy correction
correction_function = cross_section.EnergyCorrection.shower_energy_correction[args.correction]

with open(args.correction_params, "r") as f:
    correction_params = json.load(f)

e0 = {s : correction_function(shower_pairs[s].reco_lead_energy, **correction_params) for s in samples}
e1 = {s : correction_function(shower_pairs[s].reco_sub_energy, **correction_params) for s in samples}

mass = {s : Master.ShowerPairs.Mass(e0[s], e1[s], shower_pairs[s].reco_angle) for s in samples}
mass = {s : ak.fill_none(ak.pad_none(mass[s], 1, -1), -999, -1) for s in samples}
print(mass)

# mask = (mass > 50) & (mass < 250)
# mask = ak.flatten(mask) # 1 pi0


Plots.PlotTagged(mass["mc"], Tags.GeneratePi0Tags(samples["mc"], photonCandidates["mc"]), data2 = mass["data"], bins = 50, x_label = "Invariant mass (MeV)", x_range = [0, 500], norm = True)
Plots.DrawCutPosition(50, face = "right", arrow_length = 50)
Plots.DrawCutPosition(250, face = "left", arrow_length = 50)

Plots.PlotTagged(mass["mc"], EventSelection.GenerateTrueFinalStateTags(samples["mc"]), data2 = mass["data"], bins = 50, x_label = "Invariant mass (MeV)", x_range = [0, 500], norm = True)
Plots.DrawCutPosition(50, face = "right", arrow_length = 50)
Plots.DrawCutPosition(250, face = "left", arrow_length = 50)

masks = {s : ak.flatten((mass[s] > 50) & (mass[s] < 250)) for s in samples}

print(len(masks["mc"]))
print(ak.count(masks["mc"]))
print(ak.sum(masks["mc"]))

FilterSamples(samples, masks)
photonCandidates = {s : photonCandidates[s][masks[s]] for s in samples}

In [ ]:
df = samples["mc"].cutTable.get_table()
print(df.iloc[-1])

In [ ]:
shower_pairs = {s : Master.ShowerPairs(samples[s], shower_pair_mask = photonCandidates[s]) for s in samples}

angle = {s : ak.flatten(shower_pairs[s].reco_angle) for s in samples}

Plots.PlotTagged(angle["mc"], Tags.GeneratePi0Tags(samples["mc"], photonCandidates["mc"]), data2 = angle["data"], bins = 50, x_label = "Opening angle (rad)", x_range = [0, np.pi], norm = True)
Plots.DrawCutPosition((10 * np.pi / 180), face = "right", arrow_length = 0.5)
Plots.DrawCutPosition((80 * np.pi / 180), face = "left", arrow_length = 0.5)

masks = {s : (angle[s] > (10 * np.pi / 180)) & (angle[s] < (80 * np.pi / 180)) for s in samples}
FilterSamples(samples, masks)
# photonCandidates = {s : photonCandidates[s][masks[s]] for s in samples}

In [ ]:
samples["mc"].cutTable.get_table()

# samples["mc"].cutTable._data_changed = True
# for f in samples["mc"].cutTable:
#     print(f(samples["mc"].cutTable._init_data))

# samples["mc"].cutTable.get_masks()

In [ ]:
event_topology_tags = EventSelection.GenerateTrueFinalStateTags(samples["mc"])

for t in event_topology_tags:
    print(event_topology_tags[t].name_simple, ak.sum(event_topology_tags[t].mask), ak.count(event_topology_tags[t].mask))